In [1]:
from __future__ import print_function
import sys
from operator import add
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession\
.builder\
.appName("Assignment_2")\
.getOrCreate()


In [3]:
input_path = "/Users/Sankalp/Documents/Study/Big_Data/Join"

In [4]:
records = spark.sparkContext.textFile(input_path)

In [5]:
records.collect()

['A,X,V1',
 'A,X,V2',
 'A,Y,V3',
 'A,Y,V4',
 'A,Z,V5',
 'B,X,V6',
 'B,X,V7',
 'B,Y,V8',
 'B,P,V8']

In [6]:
tokens = records.map(lambda x: x.split(','))

In [7]:
tokens.collect()

[['A', 'X', 'V1'],
 ['A', 'X', 'V2'],
 ['A', 'Y', 'V3'],
 ['A', 'Y', 'V4'],
 ['A', 'Z', 'V5'],
 ['B', 'X', 'V6'],
 ['B', 'X', 'V7'],
 ['B', 'Y', 'V8'],
 ['B', 'P', 'V8']]

In [8]:
tokens1 = tokens.map(lambda x: (x[1],[x[0], x[2]]))

In [9]:
tokens1.collect()

[('X', ['A', 'V1']),
 ('X', ['A', 'V2']),
 ('Y', ['A', 'V3']),
 ('Y', ['A', 'V4']),
 ('Z', ['A', 'V5']),
 ('X', ['B', 'V6']),
 ('X', ['B', 'V7']),
 ('Y', ['B', 'V8']),
 ('P', ['B', 'V8'])]

In [10]:
tokens2 = tokens1.groupByKey()

In [11]:
tokens2.collect()

[('X', <pyspark.resultiterable.ResultIterable at 0x1207065f8>),
 ('Y', <pyspark.resultiterable.ResultIterable at 0x120706be0>),
 ('Z', <pyspark.resultiterable.ResultIterable at 0x120706518>),
 ('P', <pyspark.resultiterable.ResultIterable at 0x1207066d8>)]

In [12]:
tokens3 = tokens2.map(lambda x: (x[0], list(x[1])))

In [13]:
tokens3.collect()

[('X', [['A', 'V1'], ['A', 'V2'], ['B', 'V6'], ['B', 'V7']]),
 ('Y', [['A', 'V3'], ['A', 'V4'], ['B', 'V8']]),
 ('Z', [['A', 'V5']]),
 ('P', [['B', 'V8']])]

In [14]:
def combine_lists(x):
    list_a = []
    list_b = []
    list_c = []
    for i in x:
        first, second = i
        if first == 'A':
            list_a.append(second)
        elif first == 'B':
            list_b.append(second)
    for a in list_a:
        for b in list_b:
            list_c.append(tuple([a, b]))
    return (list_c)

In [15]:
tokens4 = tokens3.map(lambda x: (x[0], combine_lists(x[1]))).filter(lambda x: len(x[1]) >1)

In [16]:
tokens4.collect()

[('X', [('V1', 'V6'), ('V1', 'V7'), ('V2', 'V6'), ('V2', 'V7')]),
 ('Y', [('V3', 'V8'), ('V4', 'V8')])]